# Test observability windows

In [1]:
from astropy.time import Time
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from db import TelescopeManager
from surveyplanner import SurveyPlanner, Telescope

## Helper functions

In [2]:
def get_sun_set_rise(telescope, date):
    """Derive Sun set and rise for a telescope and date."""
    
    twilight = telescope['constraints']['Twilight']['twilight']
    date_datetime = date.to_datetime()
    telescope = Telescope(
        telescope['lat']*u.rad, telescope['lon']*u.rad, telescope['height'],
        telescope['utc_offset'], name=telescope['name'])
    sun_set_date, sun_rise_date = telescope.get_sun_set_rise(date_datetime.year, date_datetime.month, date_datetime.day, twilight)
    sun_set_utc = np.mod(sun_set_date.mjd, 1) * 24 - 24
    sun_rise_utc = np.mod(sun_rise_date.mjd, 1) * 24
    
    return sun_set_date, sun_rise_date, sun_set_utc, sun_rise_utc

In [3]:
def query_fields(telescope, date):
    """Query observable fields for a specific telescope and date."""
    
    fields = planner.query_fields(observable_night=date, telescope=telescope['name'])
    fields = pd.DataFrame(fields)
    fields = fields.loc[:, ('field_id', 'date_start', 'date_stop', 'duration')]

    date_start = Time(fields['date_start'].to_list())
    mjd_start = date_start.mjd
    utc_start = (mjd_start - np.floor(mjd_start.max())) * 24
    fields['date_start'] = date_start
    fields.insert(3, 'mjd_start', mjd_start)
    fields.insert(4, 'utc_start', utc_start)

    date_stop = Time(fields['date_stop'].to_list())
    mjd_stop = date_stop.mjd
    utc_stop = (mjd_stop - np.floor(mjd_stop.max())) * 24
    fields['date_stop'] = date_stop
    fields.insert(4, 'mjd_stop', mjd_stop)
    fields.insert(6, 'utc_stop', utc_stop)

    return fields

In [4]:
def plot_hist(fields):
    """Plot histograms of start and stop UTC."""
    
    __, ax = plt.subplots(1, 2, figsize=(12, 5))
    
    # plot histograms:
    sns.histplot(data=fields, x='utc_start', ax=ax[0])
    sns.histplot(data=fields, x='utc_stop', ax=ax[1])
    
    # add limits:
    for a in ax:
        a.axvline(sun_set_utc, linestyle='--', lw=2, color='orange')
        a.axvline(sun_rise_utc, linestyle='-.', lw=2, color='orange')

In [5]:
def check_times(fields, sun_set_utc, sun_rise_utc, verbose=1):
    """Check if start and stop times are within the night time."""
    
    start_too_early = np.nonzero(np.logical_and(fields['mjd_start'].values < sun_set_date.mjd, ~np.isclose(fields['mjd_start'].values, sun_set_date.mjd)))[0]
    start_too_late = np.nonzero(np.logical_and(fields['mjd_start'].values > sun_rise_date.mjd, ~np.isclose(fields['mjd_start'].values, sun_rise_date.mjd)))[0]
    stop_too_early = np.nonzero(np.logical_and(fields['mjd_stop'].values < sun_set_date.mjd, ~np.isclose(fields['mjd_stop'].values, sun_set_date.mjd)))[0]
    stop_too_late = np.nonzero(np.logical_and(fields['mjd_stop'].values > sun_rise_date.mjd, ~np.isclose(fields['mjd_stop'].values, sun_rise_date.mjd)))[0]
    
    problem_detected = False
    
    if start_too_early.shape[0]:
        problem_detected = True
        if verbose:
            print(f'Start time before Sun set: {start_too_early.shape[0]:5d} cases')
        
    if start_too_late.shape[0]:
        problem_detected = True
        if verbose:
            print(f'Start time after Sun rise:  {start_too_late.shape[0]:5d} cases')
    
    if stop_too_early.shape[0]:
        problem_detected = True
        if verbose:
            print(f'Stop time before Sun set:  {stop_too_early.shape[0]:5d} cases')
        
    if stop_too_late.shape[0]:
        problem_detected = True
        if verbose:
            print(f'Stop time after Sun rise:   {stop_too_late.shape[0]:5d} cases')
        
    if verbose and not problem_detected:
        print('All good')
        
    return start_too_early, start_too_late, stop_too_early, stop_too_late, problem_detected

## Test setup

In [6]:
db_name = 'test_planner.sqlite3'

Survey planner:

In [7]:
planner = SurveyPlanner(db_name)

Get telescopes:

In [8]:
db = TelescopeManager(db_name)
telescopes = db.get_telescopes(constraints=True)

Use fixed dates:

In [9]:
dates = Time([f'2024-{month:02d}-15' for month in range(1, 13)])

Or random dates:

In [10]:
n = 10
dates = Time('2024-01-01') + np.random.randint(0, 365, n) * u.d
dates = dates.sort()
dates

<Time object: scale='utc' format='iso' value=['2024-02-02 00:00:00.000' '2024-03-31 00:00:00.000'
 '2024-06-13 00:00:00.000' '2024-07-08 00:00:00.000'
 '2024-07-13 00:00:00.000' '2024-08-04 00:00:00.000'
 '2024-08-08 00:00:00.000' '2024-11-06 00:00:00.000'
 '2024-11-24 00:00:00.000' '2024-12-23 00:00:00.000']>

## Systematic test

In [11]:
problem_detected = 0

for date in dates:
    print(date.iso[:10])
    
    for telescope in telescopes:
        sun_set_date, sun_rise_date, sun_set_utc, sun_rise_utc = get_sun_set_rise(telescope, date)
        print('Sun set (UTC): ', sun_set_date.iso)
        print('Sun rise (UTC):', sun_rise_date.iso)
        
        fields = query_fields(telescope, date)
        __, __, __, __, problem = check_times(fields, sun_set_utc, sun_rise_utc, verbose=1)
        problem_detected += problem
        print()
    
    print()

if problem_detected:
    print('All test dates: Some problems detected!')
else:
    print('All test dates: No problems detected. All good!')

2024-02-02
Telescope Skinakas created.
Sun set (UTC):  2024-02-02 16:46:03.355
Sun rise (UTC): 2024-02-03 04:21:40.818
All good

Telescope SAAO created.
Sun set (UTC):  2024-02-02 18:37:00.097
Sun rise (UTC): 2024-02-03 03:04:11.029
All good


2024-03-31
Telescope Skinakas created.
Sun set (UTC):  2024-03-31 17:36:53.028
Sun rise (UTC): 2024-04-01 03:11:10.783
All good

Telescope SAAO created.
Sun set (UTC):  2024-03-31 17:26:01.540
Sun rise (UTC): 2024-04-01 03:55:38.302
All good


2024-06-13
Telescope Skinakas created.
Sun set (UTC):  2024-06-13 18:42:34.345
Sun rise (UTC): 2024-06-14 01:58:44.394
All good

Telescope SAAO created.
Sun set (UTC):  2024-06-13 16:35:55.575
Sun rise (UTC): 2024-06-14 04:38:14.119
All good


2024-07-08
Telescope Skinakas created.
Sun set (UTC):  2024-07-08 18:43:18.787
Sun rise (UTC): 2024-07-09 02:08:10.732
All good

Telescope SAAO created.
Sun set (UTC):  2024-07-08 16:43:12.145
Sun rise (UTC): 2024-07-09 04:40:40.849
All good


2024-07-13
Telescope Ski

## Check issues

In [12]:
telescope = telescopes[0]
date = Time('2024-03-20')

fields = query_fields(telescope, date)
sun_set_date, sun_rise_date, sun_set_utc, sun_rise_utc = get_sun_set_rise(telescope, date)
print('Sun set (UTC): ', sun_set_date.iso)
print('Sun rise (UTC):', sun_rise_date.iso)
check_times(fields, sun_set_utc, sun_rise_utc, verbose=1);

Telescope Skinakas created.
Sun set (UTC):  2024-03-20 17:27:17.678
Sun rise (UTC): 2024-03-21 03:27:23.281
All good


In [13]:
sel = fields['mjd_start'] > sun_rise_date.mjd

In [14]:
fields.loc[sel, ('date_start', 'date_stop', 'utc_start', 'utc_stop')]

,date_start,date_stop,utc_start,utc_stop
